In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics.pairwise import cosine_similarity

In [38]:

books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')

In [39]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [40]:
books.isnull().sum()

id                              0
book_id                         0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [41]:
books['isbn13'] = books['isbn13'].fillna(books['isbn13'].median())

In [42]:
books['original_publication_year'] = books['original_publication_year'].fillna(books['original_publication_year'].median())

In [43]:
books['isbn'] = books['isbn'].fillna('Unknown')

In [44]:
books['original_title'] = books['original_title'].fillna('Unknown')

In [45]:
books['language_code'] = books['language_code'].fillna('Unknown')

In [46]:
books.isnull().sum()

id                           0
book_id                      0
best_book_id                 0
work_id                      0
books_count                  0
isbn                         0
isbn13                       0
authors                      0
original_publication_year    0
original_title               0
title                        0
language_code                0
average_rating               0
ratings_count                0
work_ratings_count           0
work_text_reviews_count      0
ratings_1                    0
ratings_2                    0
ratings_3                    0
ratings_4                    0
ratings_5                    0
image_url                    0
small_image_url              0
dtype: int64

In [47]:
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
title_matrix = tfidf.fit_transform(books['title'])

In [50]:
scaler = MinMaxScaler()
books[['original_publication_year']] = scaler.fit_transform(books[['original_publication_year']])

print("Data preprocessing complete.") 

Data preprocessing complete.


In [51]:
# Keep the most recent rating for each user-book pair
ratings_latest = ratings.sort_values('timestamp').drop_duplicates(subset=['user_id', 'book_id'], keep='last')

# Pivot the data to create the user-item matrix
user_item_matrix = ratings_latest.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

# Now you can proceed with your recommendation function
recommended_books = hybrid_recommendations(user_id)
print(recommended_books)


KeyError: 'timestamp'

In [ ]:
#2
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_item_matrix)

In [ ]:
user_id = 5

In [ ]:
k = 5

In [ ]:
user_index = user_item_matrix.index.get_loc(user_id)  # Get the index of the user
distances, indices = knn_model.kneighbors(user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=k+1)

In [ ]:
similar_users = indices.flatten()[1:]

In [ ]:
user_recommendations = user_item_matrix.iloc[similar_users].mean(axis=0).sort_values(ascending=False)
print("Top recommended books:\n", user_recommendations.head(10))

In [ ]:
#3
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(books['title'])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
def get_content_recommendations(book_id, cosine_sim=cosine_sim):
    idx = books.index[books['book_id'] == book_id][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_books_indices = [i[0] for i in sim_scores[1:11]]
    return books['title'].iloc[top_books_indices]

In [ ]:
book_id = 1
print("Books similar to the selected book:\n", get_content_recommendations(book_id))

In [ ]:
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')

In [ ]:
ratings = ratings.drop_duplicates(subset=['user_id', 'book_id'])

In [ ]:
#4
def hybrid_recommendations(user_id, num_recommendations=5, weight_collab=0.5, weight_content=0.5):
    collab_scores = user_user_sim[user_id] @ user_item_matrix  # Collaborative filtering scores for books
    content_scores = cosine_sim @ user_item_matrix.loc[user_id]  # Content-based scores based on book similarity
    final_scores = (weight_collab * collab_scores) + (weight_content * content_scores)
    recommendations = final_scores.sort_values(ascending=False).head(num_recommendations)
    recommended_books = books.loc[recommendations.index]
    return recommended_books[['title', 'authors', 'average_rating']]

In [ ]:
user_item_matrix = ratings.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)

In [ ]:
user_user_sim = cosine_similarity(user_item_matrix)
user_user_sim = pd.DataFrame(user_user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

In [ ]:
user_id = 10  
recommended_books = hybrid_recommendations(user_id)
print(recommended_books)